In [5]:
import os
import sys
import pandas
import numpy
import shutil
import cProfile, pstats, io

%load_ext cython

In [2]:
from algorithm_tester.tester_logic import run_algorithms_for_file
from algorithm_tester.tester_dataclasses import AlgTesterContext
from algorithm_tester.helpers import create_path

In [3]:
def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [4]:
@profile
def profileRunAlgorithms(context, input_file):
    run_algorithms_for_file(context, input_file)

out_dir: str = ".tmp_out_dir"
context: AlgTesterContext = AlgTesterContext(
    algorithms=["SA"], parser="KnapsackParser", communicators=[],
    check_time=True, time_retries=1, max_num=None,
    extra_options={"init_temperature":2500, "min_temperature":1, "cooling":0.995, "cycles":50},
    input_dir="../data/NK", output_dir=out_dir
    )
    
create_path(out_dir)
with open("../data/Profile/Profile_40_inst.dat") as input_file:
    profileRunAlgorithms(context, input_file)
    
shutil.rmtree(out_dir)

Running output for: Profile_40_inst_SA_sol.dat. Started 19:29:51 26.12.
         574711 function calls in 0.885 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.885    0.885 <ipython-input-4-b22bf7c8de01>:1(profileRunAlgorithms)
        1    0.002    0.002    0.885    0.885 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:67(run_algorithms_for_file)
        2    0.000    0.000    0.882    0.441 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:34(get_instance_file_results)
        1    0.000    0.000    0.882    0.882 /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/timeit.py:162(timeit)
        1    0.000    0.000    0.882    0.882 <timeit-src>:2(inner)
        1    0.000    0.000    0.882    0.882 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logi

In [9]:
high = 1000
cnt = 1000000

In [26]:
%%cython

cpdef void ctest1(float init_temp=2500, float end_temp=1.0, float cooling_coef=0.995, int cycles = 50):
    cdef float curr_temp = init_temp
    cdef i, d
    
    d = 0
    for i in range(cycles):
        d += i
    
    while curr_temp > end_temp:
        curr_temp *= cooling_coef

In [27]:
@profile
def test2(init_temp: float = 2500, end_temp: float = 1.0, cooling_coef: float = 0.995, cycles: int = 50):
    curr_temp: float = init_temp
        
    d = 0
    
    while curr_temp > end_temp:
        for i in range(cycles):
            d += i
        curr_temp *= cooling_coef
        
@profile
def test1(init_temp: float = 2500, end_temp: float = 1.0, cooling_coef: float = 0.995, cycles: int = 50):
    ctest1(init_temp, end_temp, cooling_coef, cycles)

In [29]:
test2(cycles=5000)

         2 function calls in 0.483 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.483    0.483    0.483    0.483 <ipython-input-27-c2625ed93909>:1(test2)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}





In [30]:
test1(cycles=5000)

         3 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-27-c2625ed93909>:12(test1)
        1    0.000    0.000    0.000    0.000 {built-in method _cython_magic_a6659c563660b47725d51814e45e8c0f.ctest1}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}



